# Data Quality Rules - Cricket Commentary Data Pipeline
####This notebook defines and executes **comprehensive data quality checks** on the bronze layer tables
####ingested via Auto Loader. It runs after every pipeline execution and logs results to a DQ audit table.

##**Tables Validated:**
####`T20_catalog.bronze.match_events` (ball-by-ball CSV)
####`T20_catalog.bronze.match_metadata` (match-level JSON)
####`T20_catalog.bronze.match_players` (player-level CSV)

## 1. Imports & Configuration

In [0]:
# ── Job Parameters ────────────────────────────────────────────────────────────
# Default values are used during interactive runs.
# Databricks Job overrides these at runtime via the Parameters section.
# Key names here must match exactly what the Job defines.

dbutils.widgets.text(
    "catalog_name", "T20_catalog_dev",
    "Catalog Name"
)

In [0]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import (
    col, count, when, isnull, isnan, lit, sum as _sum, avg, min as _min, max as _max,
    length, trim, regexp_extract, current_timestamp, to_timestamp, datediff,
    countDistinct, expr, upper, lower, array_contains, size, coalesce
)
from pyspark.sql.types import IntegerType, LongType, DoubleType, StringType
from datetime import datetime, timedelta, timezone

# Unity Catalog Configuration
CATALOG_NAME  = dbutils.widgets.get("catalog_name")
SCHEMA_NAME = "bronze"
FULL_SCHEMA = f"{CATALOG_NAME}.{SCHEMA_NAME}"

# DQ Audit table
DQ_AUDIT_TABLE = f"{FULL_SCHEMA}.dq_audit_log"

# Tables to validate
MATCH_EVENTS_TABLE = f"{FULL_SCHEMA}.match_events"
MATCH_METADATA_TABLE = f"{FULL_SCHEMA}.match_metadata"
MATCH_PLAYERS_TABLE = f"{FULL_SCHEMA}.match_players"

# Timestamp for this run
run_timestamp = datetime.now(timezone.utc)
run_id = run_timestamp.strftime("%Y%m%d_%H%M%S")

print(f"DQ Run ID: {run_id}")
print(f"Run Timestamp: {run_timestamp}")

## 2. DQ Audit Table Setup & Helper Functions

In [0]:
# Create audit table if not exists
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {DQ_AUDIT_TABLE} (
        run_id              STRING      COMMENT 'Unique identifier for this DQ run',
        run_timestamp       TIMESTAMP   COMMENT 'When the DQ check was executed',
        table_name          STRING      COMMENT 'Fully qualified table name being checked',
        rule_category       STRING      COMMENT 'Category: completeness, validity, uniqueness, consistency, timeliness, accuracy, integrity',
        rule_name           STRING      COMMENT 'Descriptive name of the DQ rule',
        rule_description    STRING      COMMENT 'Detailed description of what the rule checks',
        total_records       LONG        COMMENT 'Total records in scope',
        passed_records      LONG        COMMENT 'Records that passed the check',
        failed_records      LONG        COMMENT 'Records that failed the check',
        pass_percentage     DOUBLE      COMMENT 'Percentage of records that passed',
        status              STRING      COMMENT 'PASS, WARN, FAIL based on thresholds',
        threshold_pct       DOUBLE      COMMENT 'Minimum acceptable pass percentage',
        details             STRING      COMMENT 'Additional details or sample failures'
    )
    USING DELTA
    COMMENT 'Data quality audit log for IPL cricket pipeline'
""")

print(f"✓ DQ audit table ready: {DQ_AUDIT_TABLE}")

In [0]:
def log_dq_result(table_name, rule_category, rule_name, rule_description,
                  total_records, passed_records, threshold_pct=100.0, details=""):
    """Log a single DQ check result to the audit table."""
    
    failed_records = total_records - passed_records
    pass_pct = (passed_records / total_records * 100) if total_records > 0 else 0.0
    
    if pass_pct >= threshold_pct:
        status = "PASS"
    elif pass_pct >= (threshold_pct - 5):
        status = "WARN"
    else:
        status = "FAIL"
    
    icon = "✓" if status == "PASS" else ("⚠" if status == "WARN" else "✗")
    print(f"  {icon} [{status}] {rule_name}: {pass_pct:.2f}% passed ({failed_records} failures)")
    
    row = spark.createDataFrame([{
        "run_id": run_id,
        "run_timestamp": run_timestamp,
        "table_name": table_name,
        "rule_category": rule_category,
        "rule_name": rule_name,
        "rule_description": rule_description,
        "total_records": int(total_records),
        "passed_records": int(passed_records),
        "failed_records": int(failed_records),
        "pass_percentage": round(pass_pct, 2),
        "status": status,
        "threshold_pct": threshold_pct,
        "details": details[:500]  # truncate long details
    }])
    
    row.write.mode("append").saveAsTable(DQ_AUDIT_TABLE)
    return status


def get_null_count(df, column):
    """Count nulls, empty strings, and 'null' string values."""
    return df.filter(
        col(column).isNull() | 
        (trim(col(column)) == "") | 
        (lower(trim(col(column))) == "null") |
        (lower(trim(col(column))) == "none") |
        (lower(trim(col(column))) == "n/a")
    ).count()

## 3. Load Tables

In [0]:
df_events = spark.table(MATCH_EVENTS_TABLE)
df_metadata = spark.table(MATCH_METADATA_TABLE)
df_players = spark.table(MATCH_PLAYERS_TABLE)

events_count = df_events.count()
metadata_count = df_metadata.count()
players_count = df_players.count()

print(f"match_events:   {events_count:,} rows")
print(f"match_metadata: {metadata_count:,} rows")
print(f"match_players:  {players_count:,} rows")

## 4. DATA QUALITY RULES

#### 4.1 COMPLETENESS RULES
 _Checks that required fields are populated (not null, not empty)._

In [0]:
print("=" * 80)
print("CATEGORY: COMPLETENESS")
print("=" * 80)

# ──────────────────────────────────────────────
# MATCH_EVENTS - Completeness
# ──────────────────────────────────────────────
print(f"\n--- {MATCH_EVENTS_TABLE} ---")

# Rule C-E01: matchid must not be null
null_matchid = df_events.filter(col("matchid").isNull()).count()
log_dq_result(MATCH_EVENTS_TABLE, "completeness", "C-E01: matchid_not_null",
    "matchid must be populated for every event row",
    events_count, events_count - null_matchid, 100.0)

# Rule C-E02: ball must not be null
null_ball = df_events.filter(col("ball").isNull() | (trim(col("ball")) == "")).count()
log_dq_result(MATCH_EVENTS_TABLE, "completeness", "C-E02: ball_not_null",
    "ball identifier (e.g., 0.1, 1.2) must be populated",
    events_count, events_count - null_ball, 100.0)

# Rule C-E03: innings must not be null
null_innings = df_events.filter(col("innings").isNull() | (trim(col("innings")) == "")).count()
log_dq_result(MATCH_EVENTS_TABLE, "completeness", "C-E03: innings_not_null",
    "innings must be populated (innings_1 or innings_2)",
    events_count, events_count - null_innings, 100.0)

# Rule C-E04: commentary must not be null
null_commentary = get_null_count(df_events, "commentary")
log_dq_result(MATCH_EVENTS_TABLE, "completeness", "C-E04: commentary_not_null",
    "Ball-by-ball commentary text should be present",
    events_count, events_count - null_commentary, 75.0)

# Rule C-E05: bowler must not be null
null_bowler = get_null_count(df_events, "bowler")
log_dq_result(MATCH_EVENTS_TABLE, "completeness", "C-E05: bowler_not_null",
    "Bowler name should be present for every delivery",
    events_count, events_count - null_bowler, 95.0)

# Rule C-E06: batsman must not be null
null_batsman = get_null_count(df_events, "batsman")
log_dq_result(MATCH_EVENTS_TABLE, "completeness", "C-E06: batsman_not_null",
    "Batsman name should be present for every delivery",
    events_count, events_count - null_batsman, 95.0)

# Rule C-E07: event must not be null
null_event = get_null_count(df_events, "event")
log_dq_result(MATCH_EVENTS_TABLE, "completeness", "C-E08: event_not_null",
    "Event type (run, wicket, wide, etc.) should be populated",
    events_count, events_count - null_event, 90.0)

# Rule C-E08: source_file must not be null (Auto Loader metadata)
null_source = df_events.filter(col("source_file").isNull()).count()
log_dq_result(MATCH_EVENTS_TABLE, "completeness", "C-E09: source_file_not_null",
    "Auto Loader source file path must be captured",
    events_count, events_count - null_source, 100.0)

# Rule C-E09: load_timestamp must not be null
null_load_ts = df_events.filter(col("load_timestamp").isNull()).count()
log_dq_result(MATCH_EVENTS_TABLE, "completeness", "C-E10: load_timestamp_not_null",
    "Load timestamp from Auto Loader must be present",
    events_count, events_count - null_load_ts, 100.0)

    # ──────────────────────────────────────────────
# MATCH_METADATA - Completeness
# ──────────────────────────────────────────────
print(f"\n--- {MATCH_METADATA_TABLE} ---")

# Rule C-M01: matchid not null
null_m_matchid = df_metadata.filter(col("matchid").isNull()).count()
log_dq_result(MATCH_METADATA_TABLE, "completeness", "C-M01: matchid_not_null",
    "matchid must be populated for every metadata record",
    metadata_count, metadata_count - null_m_matchid, 100.0)

# Rule C-M02: ground not null
null_ground = get_null_count(df_metadata, "ground")
log_dq_result(MATCH_METADATA_TABLE, "completeness", "C-M02: ground_not_null",
    "Match venue/stadium must be populated",
    metadata_count, metadata_count - null_ground, 100.0)

# Rule C-M03: series not null
null_series = get_null_count(df_metadata, "series")
log_dq_result(MATCH_METADATA_TABLE, "completeness", "C-M03: series_not_null",
    "Series/tournament name must be populated",
    metadata_count, metadata_count - null_series, 100.0)

# Rule C-M04: season not null
null_season = get_null_count(df_metadata, "season")
log_dq_result(MATCH_METADATA_TABLE, "completeness", "C-M04: season_not_null",
    "Season/year must be populated",
    metadata_count, metadata_count - null_season, 100.0)

# Rule C-M05: match_referee not null
null_match_referee = get_null_count(df_metadata, "match_referee")
log_dq_result(MATCH_METADATA_TABLE, "completeness", "C-M05: match_referee_not_null",
    "Match winner must be populated (unless no result)",
    metadata_count, metadata_count - null_match_referee, 95.0)

# Rule C-M06: toss not null
null_toss = get_null_count(df_metadata, "toss")
log_dq_result(MATCH_METADATA_TABLE, "completeness", "C-M06: toss_not_null",
    "Toss result must be populated",
    metadata_count, metadata_count - null_toss, 100.0)

# Rule C-M07: first_innings_score not null
null_fis = get_null_count(df_metadata, "first_innings")
log_dq_result(MATCH_METADATA_TABLE, "completeness", "C-M07: first_innings_not_null",
    "First innings score must be populated for completed matches",
    metadata_count, metadata_count - null_fis, 95.0)

# Rule C-M08: second_innings_score not null
null_sis = get_null_count(df_metadata, "second_innings")
log_dq_result(MATCH_METADATA_TABLE, "completeness", "C-M08: second_innings_not_null",
    "Second innings score must be populated for completed matches",
    metadata_count, metadata_count - null_sis, 95.0)

# Rule C-M09: source_file not null
null_m_source = df_metadata.filter(col("source_file").isNull()).count()
log_dq_result(MATCH_METADATA_TABLE, "completeness", "C-M09: source_file_not_null",
    "Source file path from Auto Loader must be captured",
    metadata_count, metadata_count - null_m_source, 100.0)

# Rule C-M10: umpire not null
null_umpires = get_null_count(df_metadata, "umpires")
log_dq_result(MATCH_METADATA_TABLE, "completeness", "C-M10: umpires_not_null",
    "Match winner must be populated (unless no result)",
    metadata_count, metadata_count - null_umpires, 95.0)

# Rule C-M11: tv_umpire not null
null_tv_umpire = get_null_count(df_metadata, "tv_umpire")
log_dq_result(MATCH_METADATA_TABLE, "completeness", "C-M11: tv_umpire_not_null",
    "Match winner must be populated (unless no result)",
    metadata_count, metadata_count - null_tv_umpire, 95.0)

# Rule C-M12: match_number not null
null_match_number = get_null_count(df_metadata, "match_number")
log_dq_result(MATCH_METADATA_TABLE, "completeness", "C-M12: match_number_not_null",
    "Match winner must be populated (unless no result)",
    metadata_count, metadata_count - null_match_number, 95.0)
# ──────────────────────────────────────────────
# MATCH_PLAYERS - Completeness
# ──────────────────────────────────────────────
print(f"\n--- {MATCH_PLAYERS_TABLE} ---")

# Rule C-P01: matchid not null
null_p_matchid = df_players.filter(col("matchid").isNull()).count()
log_dq_result(MATCH_PLAYERS_TABLE, "completeness", "C-P01: matchid_not_null",
    "matchid must be populated for every player record",
    players_count, players_count - null_p_matchid, 100.0)

# Rule C-P02: player_name not null
null_p_player_name = df_players.filter(col("matchid").isNull()).count()
log_dq_result(MATCH_PLAYERS_TABLE, "completeness", "C-P02: player_name_not_null",
    "matchid must be populated for every player record",
    players_count, players_count - null_p_player_name, 100.0)

# Rule C-P03: team not null
null_p_team = df_players.filter(col("matchid").isNull()).count()
log_dq_result(MATCH_PLAYERS_TABLE, "completeness", "C-P03: team_not_null",
    "matchid must be populated for every player record",
    players_count, players_count - null_p_team, 100.0)

# Rule C-P04: innings not null
if "innings" in df_players.columns:
    null_p_innings = get_null_count(df_players, "innings")
    log_dq_result(MATCH_PLAYERS_TABLE, "completeness", "C-P04: innings_not_null",
        "Innings assignment must be populated for every player",
        players_count, players_count - null_p_innings, 100.0)

### 4.2 VALIDITY / FORMAT RULES
_Checks that values conform to expected formats, ranges, and domains._

In [0]:
# COMMAND ----------

print("=" * 80)
print("CATEGORY: VALIDITY")
print("=" * 80)

# ──────────────────────────────────────────────
# MATCH_EVENTS - Validity
# ──────────────────────────────────────────────
print(f"\n--- {MATCH_EVENTS_TABLE} ---")

# Rule V-E01: matchid must be a positive integer
invalid_matchid = df_events.filter(
    (col("matchid").isNull()) | (col("matchid") <= 0)
).count()
log_dq_result(MATCH_EVENTS_TABLE, "validity", "V-E01: matchid_positive_integer",
    "matchid must be a positive integer value",
    events_count, events_count - invalid_matchid, 100.0)

# Rule V-E02: ball must follow cricket over.ball format (e.g., 0.1, 19.6)
invalid_ball = df_events.filter(
    ~col("ball").rlike(r"^\d{1,2}\.\d{1}$")
).count()
log_dq_result(MATCH_EVENTS_TABLE, "validity", "V-E02: ball_format_valid",
    "ball must follow over.ball_number format (e.g., 0.1, 15.3, 19.6)",
    events_count, events_count - invalid_ball, 98.0,
    "Expected format: D.D or DD.D where over 0-19, ball 1-6+")

# Rule V-E03: over number (from ball) must be 0-19 for T20
df_events_with_over = df_events.withColumn(
    "over_num", regexp_extract(col("ball"), r"^(\d{1,2})\.", 1).cast("int")
)
invalid_over = df_events_with_over.filter(
    (col("over_num").isNull()) | (col("over_num") < 0) | (col("over_num") > 300)
).count()
log_dq_result(MATCH_EVENTS_TABLE, "validity", "V-E03: over_number_range_0_19",
    "Over number extracted from ball must be between 0 and 300 (ODI,T20 and test format)",
    events_count, events_count - invalid_over, 98.0)

# Rule V-E04: ball number within over must be 1-9 (6 legal + extras)
df_events_with_ball_num = df_events.withColumn(
    "ball_in_over", regexp_extract(col("ball"), r"\.(\d{1})$", 1).cast("int")
)
invalid_ball_num = df_events_with_ball_num.filter(
    (col("ball_in_over").isNull()) | (col("ball_in_over") < 1) | (col("ball_in_over") > 6)
).count()
log_dq_result(MATCH_EVENTS_TABLE, "validity", "V-E04: ball_in_over_range_1_6",
    "Ball number within over should be 1-6 (extras will have the same ball number)",
    events_count, events_count - invalid_ball_num, 98.0)

# Rule V-E05: bowler name should be alphabetic (allow spaces, dots, apostrophes)
invalid_bowler_name = df_events.filter(
    col("bowler").isNotNull() & ~col("bowler").rlike(r"^[A-Za-z\s\.\'\-]+$")
).count()
log_dq_result(MATCH_EVENTS_TABLE, "validity", "V-E06: bowler_name_format",
    "Bowler name should contain only letters, spaces, dots, hyphens, apostrophes",
    events_count, events_count - invalid_bowler_name, 80.0)

# Rule V-E06: batsman name should be alphabetic
invalid_batsman_name = df_events.filter(
    col("batsman").isNotNull() & ~col("batsman").rlike(r"^[A-Za-z\s\.\'\-]+$")
).count()
log_dq_result(MATCH_EVENTS_TABLE, "validity", "V-E07: batsman_name_format",
    "Batsman name should contain only letters, spaces, dots, hyphens, apostrophes",
    events_count, events_count - invalid_batsman_name, 80.0)

# Rule V-E07: source_file should contain matchid= pattern (ADLS path validation)
invalid_source_path = df_events.filter(
    ~col("source_file").contains("matchid=")
).count()
log_dq_result(MATCH_EVENTS_TABLE, "validity", "V-E09: source_file_contains_matchid",
    "Source file path must contain 'matchid=' partition pattern from ADLS",
    events_count, events_count - invalid_source_path, 100.0)

# Rule V-E08: matchid extracted from path should match data matchid
df_events_path_check = df_events.withColumn(
    "path_matchid",
    regexp_extract(col("source_file"), r"matchid=(\d+)", 1).cast("int")
)
mismatched_matchid = df_events_path_check.filter(
    col("matchid") != col("path_matchid")
).count()
log_dq_result(MATCH_EVENTS_TABLE, "validity", "V-E10: matchid_matches_source_path",
    "matchid in data must match matchid extracted from ADLS file path",
    events_count, events_count - mismatched_matchid, 100.0)

# ──────────────────────────────────────────────
# MATCH_METADATA - Validity
# ──────────────────────────────────────────────
print(f"\n--- {MATCH_METADATA_TABLE} ---")

# Rule V-M01: matchid must be positive integer
invalid_m_matchid = df_metadata.filter(
    (col("matchid").isNull()) | (col("matchid") <= 0)
).count()
log_dq_result(MATCH_METADATA_TABLE, "validity", "V-M01: matchid_positive_integer",
    "matchid must be a positive integer",
    metadata_count, metadata_count - invalid_m_matchid, 100.0)

# Rule V-M02: batting_first != batting_second (different teams)
same_teams = df_metadata.filter(
    col("first_innings").isNotNull() & 
    col("second_innings").isNotNull() &
    (trim(col("first_innings")) == trim(col("second_innings")))
).count()
log_dq_result(MATCH_METADATA_TABLE, "validity", "V-M07: teams_are_different",
    "batting_first and batting_second must be different teams",
    metadata_count, metadata_count - same_teams, 100.0)

# Rule V-M03: season should be a valid year (2008-2030 for IPL)
invalid_season = df_metadata.filter(
    col("season").isNotNull() & (
        ~col("season").rlike(r"^\d{4}(/\d{2})?$") |
        (regexp_extract(col("season"), r"^(\d{4})", 1).cast("int") < 2008) |
        (regexp_extract(col("season"), r"^(\d{4})", 1).cast("int") > 2030)
    )
).count()
log_dq_result(MATCH_METADATA_TABLE, "validity", "V-M02: season_valid_year",
    "Season must be a valid year between 2008-2030 for IPL",
    metadata_count, metadata_count - invalid_season, 100.0)

# ──────────────────────────────────────────────
# MATCH_PLAYERS - Validity
# ──────────────────────────────────────────────
print(f"\n--- {MATCH_PLAYERS_TABLE} ---")

# Rule V-P01: matchid must be positive integer
invalid_p_matchid = df_players.filter(
    (col("matchid").isNull()) | (col("matchid") <= 0)
).count()
log_dq_result(MATCH_PLAYERS_TABLE, "validity", "V-P01: matchid_positive_integer",
    "matchid must be a positive integer",
    players_count, players_count - invalid_p_matchid, 100.0)

# Rule V-P02: innings value must be valid
if "innings" in df_players.columns:
    invalid_p_innings = df_players.filter(
        col("innings").isNotNull() &
        ~lower(trim(col("innings"))).isin(["innings_1", "innings_2"])
    ).count()
    log_dq_result(MATCH_PLAYERS_TABLE, "validity", "V-P02: innings_value_valid",
        "innings must be 'innings_1' or 'innings_2'",
        players_count, players_count - invalid_p_innings, 100.0)



### 4.3 UNIQUENESS RULES
 _Checks for duplicate records and natural key integrity._

In [0]:
print("=" * 80)
print("CATEGORY: UNIQUENESS")
print("=" * 80)

# ──────────────────────────────────────────────
# MATCH_EVENTS - Uniqueness
# ──────────────────────────────────────────────
print(f"\n--- {MATCH_EVENTS_TABLE} ---")


# Rule U-E01: No fully duplicate rows
full_distinct = df_events.distinct().count()
log_dq_result(MATCH_EVENTS_TABLE, "uniqueness", "U-E02: no_full_duplicates",
    "There should be no fully identical rows across all columns",
    events_count, full_distinct, 99.0)

# ──────────────────────────────────────────────
# MATCH_METADATA - Uniqueness
# ──────────────────────────────────────────────
print(f"\n--- {MATCH_METADATA_TABLE} ---")

# Rule U-M01: matchid should be unique (one metadata record per match)
distinct_m_matchid = df_metadata.select("matchid").distinct().count()
log_dq_result(MATCH_METADATA_TABLE, "uniqueness", "U-M01: matchid_unique",
    "Each match should have exactly one metadata record",
    metadata_count, distinct_m_matchid, 100.0,
    f"Total: {metadata_count}, Distinct matchids: {distinct_m_matchid}")

# Rule U-M02: No fully duplicate metadata rows
full_distinct_m = df_metadata.distinct().count()
log_dq_result(MATCH_METADATA_TABLE, "uniqueness", "U-M02: no_full_duplicates",
    "There should be no fully identical metadata rows",
    metadata_count, full_distinct_m, 100.0)

# ──────────────────────────────────────────────
# MATCH_PLAYERS - Uniqueness
# ──────────────────────────────────────────────
print(f"\n--- {MATCH_PLAYERS_TABLE} ---")

# Rule U-P01: Each player appears once per (matchid, innings)
player_col = None
for c in ["player", "player_name", "name"]:
    if c in df_players.columns:
        player_col = c
        break

if player_col and "innings" in df_players.columns:
    total_players = players_count
    distinct_players = df_players.select("matchid", "innings", player_col).distinct().count()
    log_dq_result(MATCH_PLAYERS_TABLE, "uniqueness", "U-P01: player_per_match_innings_unique",
        f"Each ({player_col}, matchid, innings) combination should be unique",
        total_players, distinct_players, 99.0)

# Rule U-P02: No fully duplicate player rows
full_distinct_p = df_players.distinct().count()
log_dq_result(MATCH_PLAYERS_TABLE, "uniqueness", "U-P02: no_full_duplicates",
    "There should be no fully identical player rows",
    players_count, full_distinct_p, 100.0)

### 4.4 CONSISTENCY / CROSS-TABLE INTEGRITY RULES
_Checks referential integrity and logical consistency across tables._

In [0]:
# COMMAND ----------

print("=" * 80)
print("CATEGORY: CONSISTENCY / INTEGRITY")
print("=" * 80)

# Rule I-01: Every matchid in match_events must exist in match_metadata
events_matchids = df_events.select("matchid").distinct()
metadata_matchids = df_metadata.select("matchid").distinct()
orphan_event_matchids = events_matchids.join(metadata_matchids, "matchid", "left_anti")
orphan_count = orphan_event_matchids.count()
total_event_matchids = events_matchids.count()
log_dq_result(MATCH_EVENTS_TABLE, "integrity", "I-01: events_matchid_in_metadata",
    "Every matchid in match_events must have a corresponding record in match_metadata",
    total_event_matchids, total_event_matchids - orphan_count, 100.0,
    f"Orphan matchids in events (no metadata): {orphan_count}")

# Rule I-02: Every matchid in match_players must exist in match_metadata
players_matchids = df_players.select("matchid").distinct()
orphan_player_matchids = players_matchids.join(metadata_matchids, "matchid", "left_anti")
orphan_p_count = orphan_player_matchids.count()
total_player_matchids = players_matchids.count()
log_dq_result(MATCH_PLAYERS_TABLE, "integrity", "I-02: players_matchid_in_metadata",
    "Every matchid in match_players must have a corresponding record in match_metadata",
    total_player_matchids, total_player_matchids - orphan_p_count, 100.0,
    f"Orphan matchids in players (no metadata): {orphan_p_count}")

# Rule I-03: Every matchid in match_metadata should have events
metadata_no_events = metadata_matchids.join(events_matchids, "matchid", "left_anti")
missing_events_count = metadata_no_events.count()
total_meta_matchids = metadata_matchids.count()
log_dq_result(MATCH_METADATA_TABLE, "integrity", "I-03: metadata_has_events",
    "Every match in metadata should have corresponding ball-by-ball events",
    total_meta_matchids, total_meta_matchids - missing_events_count, 95.0,
    f"Matches with metadata but no events: {missing_events_count}")

# Rule I-04: Every matchid in match_metadata should have players
metadata_no_players = metadata_matchids.join(players_matchids, "matchid", "left_anti")
missing_players_count = metadata_no_players.count()
log_dq_result(MATCH_METADATA_TABLE, "integrity", "I-04: metadata_has_players",
    "Every match in metadata should have corresponding player records",
    total_meta_matchids, total_meta_matchids - missing_players_count, 95.0,
    f"Matches with metadata but no players: {missing_players_count}")

# Rule I-05: matchid sets should be identical across all 3 tables
all_three_match = events_matchids.join(metadata_matchids, "matchid", "inner") \
                                  .join(players_matchids, "matchid", "inner")
all_matchids = events_matchids.union(metadata_matchids).union(players_matchids).distinct()
complete_count = all_three_match.count()
total_unique_matchids = all_matchids.count()
log_dq_result(MATCH_EVENTS_TABLE, "integrity", "I-05: all_tables_matchid_aligned",
    "All three tables should have the same set of matchids (complete data)",
    total_unique_matchids, complete_count, 95.0,
    f"Matches in all 3 tables: {complete_count}, Total unique matchids: {total_unique_matchids}")

# Rule I-06: Player count per match should be 22 (11 per team) +/- subs
if player_col:
    players_per_match = df_players.groupBy("matchid").agg(
        countDistinct(player_col).alias("player_count")
    )
    valid_player_count = players_per_match.filter(
        (col("player_count") >= 20) & (col("player_count") <= 30)
    ).count()
    total_matches_with_players = players_per_match.count()
    log_dq_result(MATCH_PLAYERS_TABLE, "consistency", "I-07: player_count_per_match",
        "Each match should have 20-30 unique players (11 per team + possible subs)",
        total_matches_with_players, valid_player_count, 90.0)



### 4.5 VOLUME / STATISTICAL RULES
 _Checks row counts, distributions, and detects anomalies._

In [0]:
print("=" * 80)
print("CATEGORY: VOLUME / STATISTICAL")
print("=" * 80)

# Rule S-01: match_events should have reasonable rows per match (100-300 for T20)
events_per_match = df_events.groupBy("matchid").count().withColumnRenamed("count", "row_count")
valid_volume = events_per_match.filter(
    (col("row_count") >= 100) & (col("row_count") <= 300)
).count()
total_matches = events_per_match.count()
log_dq_result(MATCH_EVENTS_TABLE, "volume", "S-01: events_per_match_reasonable",
    "Each T20 match should have ~120-500 ball-by-ball events (including extras)",
    total_matches, valid_volume, 90.0)

# Get stats for details
stats = events_per_match.select(
    _min("row_count").alias("min_rows"),
    _max("row_count").alias("max_rows"),
    avg("row_count").alias("avg_rows")
).first()
print(f"  Events per match: min={stats['min_rows']}, max={stats['max_rows']}, avg={stats['avg_rows']:.0f}")

# Rule S-02: Each innings should have ~01-160 deliveries
events_per_innings = df_events.groupBy("matchid", "innings").count().withColumnRenamed("count", "row_count")
valid_innings_volume = events_per_innings.filter(
    (col("row_count") >= 1) & (col("row_count") <= 160)
).count()
total_innings = events_per_innings.count()
log_dq_result(MATCH_EVENTS_TABLE, "volume", "S-02: events_per_innings_reasonable",
    "Each innings should have 30-200 deliveries (20 overs max + extras)",
    total_innings, valid_innings_volume, 90.0)

# Rule S-03: Total row count should not be zero
log_dq_result(MATCH_EVENTS_TABLE, "volume", "S-03: events_table_not_empty",
    "match_events table must contain data",
    1, 1 if events_count > 0 else 0, 100.0)

log_dq_result(MATCH_METADATA_TABLE, "volume", "S-04: metadata_table_not_empty",
    "match_metadata table must contain data",
    1, 1 if metadata_count > 0 else 0, 100.0)

log_dq_result(MATCH_PLAYERS_TABLE, "volume", "S-05: players_table_not_empty",
    "match_players table must contain data",
    1, 1 if players_count > 0 else 0, 100.0)

# Rule S-06: Distinct bowlers per innings should be ~5-8 (T20)
bowlers_per_innings = df_events.filter(col("bowler").isNotNull()).groupBy("matchid", "innings").agg(
    countDistinct("bowler").alias("bowler_count")
)
valid_bowler_count = bowlers_per_innings.filter(
    (col("bowler_count") >= 3) & (col("bowler_count") <= 10)
).count()
total_innings_bowler = bowlers_per_innings.count()
log_dq_result(MATCH_EVENTS_TABLE, "volume", "S-06: bowlers_per_innings_reasonable",
    "Each innings should have 3-10 distinct bowlers",
    total_innings_bowler, valid_bowler_count, 90.0)

# Rule S-07: Distinct batsmen per innings should be ~2-11
batsmen_per_innings = df_events.filter(col("batsman").isNotNull()).groupBy("matchid", "innings").agg(
    countDistinct("batsman").alias("batsman_count")
)
valid_batsman_count = batsmen_per_innings.filter(
    (col("batsman_count") >= 2) & (col("batsman_count") <= 11)
).count()
total_innings_batsman = batsmen_per_innings.count()
log_dq_result(MATCH_EVENTS_TABLE, "volume", "S-07: batsmen_per_innings_reasonable",
    "Each innings should have 2-11 distinct batsmen",
    total_innings_batsman, valid_batsman_count, 90.0)

### 4.6 ACCURACY / DOMAIN-SPECIFIC RULES
_Cricket-specific business logic validation._

In [0]:
print("=" * 80)
print("CATEGORY: ACCURACY / DOMAIN-SPECIFIC")
print("=" * 80)

# Rule A-01: First ball of each innings should be 0.1
first_balls = df_events.groupBy("matchid", "innings").agg(
    _min("ball").alias("first_ball")
)
valid_first_ball = first_balls.filter(col("first_ball") == "0.1").count()
total_innings_first = first_balls.count()
log_dq_result(MATCH_EVENTS_TABLE, "accuracy", "A-01: first_ball_is_0_1",
    "The first delivery of each innings should be ball 0.1",
    total_innings_first, valid_first_ball, 95.0)

# Rule A-02: Last over should not exceed 19 (0-indexed) in T20
last_overs = df_events.withColumn(
    "over_num", regexp_extract(col("ball"), r"^(\d{1,2})\.", 1).cast("int")
).groupBy("matchid", "innings").agg(
    _max("over_num").alias("last_over")
)
valid_last_over = last_overs.filter(
    (col("last_over") >= 0) & (col("last_over") <= 19)
).count()
total_innings_last = last_overs.count()
log_dq_result(MATCH_EVENTS_TABLE, "accuracy", "A-02: max_over_within_T20_limit",
    "Maximum over number should not exceed 19 (20 overs in T20, 0-indexed)",
    total_innings_last, valid_last_over, 95.0)


# Rule A-03: Commentary text should have minimum length (not truncated)
short_commentary = df_events.filter(
    col("commentary").isNotNull() & (length(trim(col("commentary"))) < 5)
).count()
log_dq_result(MATCH_EVENTS_TABLE, "accuracy", "A-03: commentary_min_length",
    "Commentary text should be at least 5 characters (not truncated or garbage)",
    events_count, events_count - short_commentary, 75.0)

# Rule A-04: No single bowler should bowl more than 4 overs in a T20 innings
bowler_overs = df_events.filter(col("bowler").isNotNull()).withColumn(
    "over_num", regexp_extract(col("ball"), r"^(\d{1,2})\.", 1).cast("int")
).groupBy("matchid", "innings", "bowler").agg(
    countDistinct("over_num").alias("overs_bowled")
)
over_limit_bowlers = bowler_overs.filter(col("overs_bowled") > 4).count()
total_bowler_spells = bowler_overs.count()
log_dq_result(MATCH_EVENTS_TABLE, "accuracy", "A-04: bowler_max_4_overs_t20",
    "No bowler should bowl more than 4 overs in a T20 innings",
    total_bowler_spells, total_bowler_spells - over_limit_bowlers, 95.0,
    f"Bowler spells exceeding 4 overs: {over_limit_bowlers}")




### 4.8 SCHEMA DRIFT DETECTION
_Detects unexpected schema changes from Auto Loader._

In [0]:
print("=" * 80)
print("CATEGORY: SCHEMA")
print("=" * 80)

# Rule SD-01: match_events expected columns
expected_events_cols = {"matchid", "ball", "innings", "event", "is_super_over","commentary", 
                         "bowler", "batsman", "source_file", "load_timestamp"}
actual_events_cols = set(df_events.columns)
missing_cols = expected_events_cols - actual_events_cols
extra_cols = actual_events_cols - expected_events_cols - {"_rescued_data"}

has_all_cols = 1 if len(missing_cols) == 0 else 0
log_dq_result(MATCH_EVENTS_TABLE, "schema", "SD-01: events_expected_columns",
    "match_events should contain all expected columns",
    len(expected_events_cols), len(expected_events_cols) - len(missing_cols), 100.0,
    f"Missing: {missing_cols if missing_cols else 'None'}, Extra: {extra_cols if extra_cols else 'None'}")

# Rule SD-02: match_metadata expected columns
expected_metadata_cols = {"matchid", "ground", "series", "season", "toss", 
                           "player_of_the_match", "umpires","tv_umpire", "source_file", "load_timestamp"}
actual_metadata_cols = set(df_metadata.columns)
missing_m_cols = expected_metadata_cols - actual_metadata_cols

log_dq_result(MATCH_METADATA_TABLE, "schema", "SD-02: metadata_expected_columns",
    "match_metadata should contain all expected columns",
    len(expected_metadata_cols), len(expected_metadata_cols) - len(missing_m_cols), 100.0,
    f"Missing: {missing_m_cols if missing_m_cols else 'None'}")

# Rule SD-03: match_players expected columns
expected_players_cols = {"matchid", "source_file", "load_timestamp"}
actual_players_cols = set(df_players.columns)
missing_p_cols = expected_players_cols - actual_players_cols

log_dq_result(MATCH_PLAYERS_TABLE, "schema", "SD-03: players_expected_columns",
    "match_players should contain all expected columns",
    len(expected_players_cols), len(expected_players_cols) - len(missing_p_cols), 100.0,
    f"Missing: {missing_p_cols if missing_p_cols else 'None'}")

# Rule SD-04: Check for _rescued_data (Auto Loader schema enforcement failures)
for table_name, df, total in [
    (MATCH_EVENTS_TABLE, df_events, events_count),
    (MATCH_METADATA_TABLE, df_metadata, metadata_count),
    (MATCH_PLAYERS_TABLE, df_players, players_count)
]:
    if "_rescued_data" in df.columns:
        rescued_count = df.filter(col("_rescued_data").isNotNull()).count()
        log_dq_result(table_name, "schema", f"SD-04: no_rescued_data_{table_name.split('.')[-1]}",
            "Auto Loader _rescued_data column should be empty (no schema mismatches)",
            total, total - rescued_count, 99.0,
            f"Records with rescued data: {rescued_count}")

## 5. DQ Summary Report

In [0]:
print("\n" + "=" * 80)
print(f"DATA QUALITY SUMMARY - Run ID: {run_id}")
print("=" * 80)

# Query audit table for this run
summary_df = spark.sql(f"""
    SELECT 
        status,
        COUNT(*) as rule_count,
        ROUND(AVG(pass_percentage), 2) as avg_pass_pct
    FROM {DQ_AUDIT_TABLE}
    WHERE run_id = '{run_id}'
    GROUP BY status
    ORDER BY status
""")
display(summary_df)

# Detailed failures
print("\n--- FAILED RULES ---")
failures_df = spark.sql(f"""
    SELECT 
        table_name,
        rule_category,
        rule_name,
        pass_percentage,
        failed_records,
        details
    FROM {DQ_AUDIT_TABLE}
    WHERE run_id = '{run_id}' AND status = 'FAIL'
    ORDER BY pass_percentage ASC
""")
display(failures_df)

# Warnings
print("\n--- WARNINGS ---")
warnings_df = spark.sql(f"""
    SELECT 
        table_name,
        rule_category,
        rule_name,
        pass_percentage,
        failed_records,
        details
    FROM {DQ_AUDIT_TABLE}
    WHERE run_id = '{run_id}' AND status = 'WARN'
    ORDER BY pass_percentage ASC
""")
display(warnings_df)

## 6. Historical DQ Trend (Optional Query)

In [0]:
# Run this to see DQ trends over time
trend_df = spark.sql(f"""
    SELECT 
        run_id,
        run_timestamp,
        COUNT(*) as total_rules,
        SUM(CASE WHEN status = 'PASS' THEN 1 ELSE 0 END) as passed,
        SUM(CASE WHEN status = 'WARN' THEN 1 ELSE 0 END) as warnings,
        SUM(CASE WHEN status = 'FAIL' THEN 1 ELSE 0 END) as failed,
        ROUND(AVG(pass_percentage), 2) as avg_pass_pct
    FROM {DQ_AUDIT_TABLE}
    GROUP BY run_id, run_timestamp
    ORDER BY run_timestamp DESC
    LIMIT 20
""")
display(trend_df)

## 7. Pipeline Gate (Optional - Fail pipeline if critical rules fail)


In [0]:
# Check for critical failures and optionally halt downstream processing
critical_failures = spark.sql(f"""
    SELECT COUNT(*) as cnt
    FROM {DQ_AUDIT_TABLE}
    WHERE run_id = '{run_id}' 
      AND status = 'FAIL'
      AND threshold_pct = 100.0
""").first()["cnt"]

if critical_failures > 0:
    msg = f"⛔ PIPELINE GATE: {critical_failures} critical DQ rule(s) failed! Review before proceeding to Silver layer."
    print(msg)
    # Uncomment to actually halt the pipeline:
    # raise Exception(msg)
else:
    print("✅ PIPELINE GATE: All critical DQ rules passed. Safe to proceed to Silver layer.")